In [ ]:
'''
# if working on laptop on local docker:
nvidia-docker run -p 4242:8888 -v ~/dl_cas/:/notebooks -p 6006:6006 -it oduerr/tf_docker:gpu_r
'''

## Imports

In [1]:
# python module imports needed in customized functions:
import numpy as np
import tensorflow as tf
import sys

In [2]:
tf.__version__

'1.4.1'

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as imgplot

In [ ]:
'''

# in case of tf1.4 keras is integrated into tf (seems not to work properly)
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Input, BatchNormalization
from tensorflow.python.keras import backend as K

from tensorflow.python.keras.callbacks import TensorBoard

'''


In [4]:
# install external keras (outside tensorflow)
!pip install keras

In [6]:
# use external keras (outside tensorflow)
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
#from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.layers.normalization import BatchNormalization

# Data read-in 

### Load small external MNIST data set when for working local on windows

In [7]:

# upload mnist_4000.pkl.gz which we have used in the DL course to home
# To be compatible with python3 and python2
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip

with gzip.open('./../mnist_4000.pkl.gz', 'rb') as f:
    if sys.version_info.major > 2:
        (X,y) = pickle.load(f, encoding='latin1')
    else:
        (X,y) = pickle.load(f)
PIXELS = len(X[0,0,0,:])

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

X = X.reshape([4000, 784])
#X = X/255 # is already normalized

print("small data before split X.shape", X.shape)
print("small data before  y.shape", y.shape) 

x_train = X[0:3000]
y_train = y[0:3000]
x_test = X[3000:4000]
y_test = y[3000:4000]


print("small data x_train.shape:", x_train.shape)
print("small data y_train.shape:",y_train.shape)
print("small data x_test.shape:",x_test.shape)
print("small data y_test.shape:",y_test.shape)

num_class= len(np.unique(y))
print("num_class:",num_class)



small data before split X.shape (4000, 784)
small data before  y.shape (4000,)
small data x_train.shape: (3000, 784)
small data y_train.shape: (3000,)
small data x_test.shape: (1000, 784)
small data y_test.shape: (1000,)
num_class: 10


### Or load full MNIST dataset directly from internet

In [8]:
'''
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

x_train = x_train.reshape(-1,784)
x_test = x_test.reshape(-1,784)

print("large data x_train.shape:", x_train.shape)
print("large data y_train.shape:",y_train.shape)
print("large data x_test.shape:",x_test.shape)
print("large data x_test.shape:",y_test.shape)
'''

'\nfrom keras.datasets import mnist\n\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\n# if images are not flatten (like in mnist) we need first to flatten them\n# now flatten images for fc ladder\n\nx_train = x_train.reshape(-1,784)\nx_test = x_test.reshape(-1,784)\n\nprint("large data x_train.shape:", x_train.shape)\nprint("large data y_train.shape:",y_train.shape)\nprint("large data x_test.shape:",x_test.shape)\nprint("large data x_test.shape:",y_test.shape)\n'

In [9]:
x_train[:200].shape

(200, 784)

In [10]:
np.mean(x_train[:,200])

-0.0042952602

In [11]:
np.std(x_train[:,200])

0.83271211

# Lets construct a fc NN (784->500->50->10) without noise and unsupervised task to get a benchmark for the loss and accuracy

In [14]:
### Relu 
name = 'relu'
model = Sequential()

model.add(Dense(500, batch_input_shape=(None, 784),
                kernel_initializer='random_uniform',
                bias_initializer='zeros'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              # optimizer='rmsprop',
              metrics=['accuracy'])

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 500)               2000      
_________________________________________________________________
activation_3 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                25050     
_________________________________________________________________
batch_normalization_4 (Batch (None, 50)                200       
_________________________________________________________________
activation_4 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                510       
Total para

# Training

In [16]:
def convertToOneHot(vector, num_classes=None):
    result = np.zeros((len(vector), num_classes), dtype='int32')
    result[np.arange(len(vector)), vector] = 1
    return result

In [17]:
tensorboard = keras.callbacks.TensorBoard(
    log_dir='tensorboard/mnist_small/' + name + '/', 
    write_graph=True,
    histogram_freq=1
)

In [18]:
history = model.fit(x_train, 
          convertToOneHot(y_train,10), 
          nb_epoch=100, 
          batch_size=128, 
          callbacks=[tensorboard],
          validation_data=[x_test, convertToOneHot(y_test,10)], verbose=2)



/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3000 samples, validate on 1000 samples
Epoch 1/100
 - 2s - loss: 0.9412 - acc: 0.7403 - val_loss: 0.4829 - val_acc: 0.8710
Epoch 2/100
 - 1s - loss: 0.3712 - acc: 0.9297 - val_loss: 0.3639 - val_acc: 0.9050
Epoch 3/100
 - 1s - loss: 0.2281 - acc: 0.9643 - val_loss: 0.3392 - val_acc: 0.9100
Epoch 4/100
 - 2s - loss: 0.1491 - acc: 0.9803 - val_loss: 0.3265 - val_acc: 0.9200
Epoch 5/100
 - 2s - loss: 0.0991 - acc: 0.9930 - val_loss: 0.3166 - val_acc: 0.9240
Epoch 6/100
 - 1s - loss: 0.0691 - acc: 0.9967 - val_loss: 0.3045 - val_acc: 0.9240
Epoch 7/100
 - 1s - loss: 0.0467 - acc: 0.9993 - val_loss: 0.2933 - val_acc: 0.9240
Epoch 8/100
 - 1s - loss: 0.0331 - acc: 0.9997 - val_loss: 0.2723 - val_acc: 0.9340
Epoch 9/100
 - 1s - loss: 0.0264 - acc: 1.0000 - val_loss: 0.2725 - val_acc: 0.9300
Epoch 10/100
 - 1s - loss: 0.0205 - acc: 1.0000 - val_loss: 0.2503 - val_acc: 0.9340
Epoch 11/100
 - 1s - loss: 0.0155 - acc: 1.0000 - val_loss: 0.2399 - val_acc: 0.9370
Epoch 12/100
 - 1s - loss:

Epoch 95/100
 - 1s - loss: 2.0140e-04 - acc: 1.0000 - val_loss: 0.2235 - val_acc: 0.9450
Epoch 96/100
 - 2s - loss: 1.9091e-04 - acc: 1.0000 - val_loss: 0.2245 - val_acc: 0.9410
Epoch 97/100
 - 1s - loss: 2.0364e-04 - acc: 1.0000 - val_loss: 0.2236 - val_acc: 0.9430
Epoch 98/100
 - 2s - loss: 1.9779e-04 - acc: 1.0000 - val_loss: 0.2243 - val_acc: 0.9420
Epoch 99/100
 - 1s - loss: 1.8371e-04 - acc: 1.0000 - val_loss: 0.2241 - val_acc: 0.9430
Epoch 100/100
 - 1s - loss: 1.8476e-04 - acc: 1.0000 - val_loss: 0.2251 - val_acc: 0.9440


# tf1.4 and external keras on gpu is slower than external keras with tf 1.0 on cpu - why???